⭕ Enter Necessary Info

In [70]:
apiname = 'transactionHistoryMultiAccounts'
start = 0
end = 9

⭕ Data to be Prepare

In [71]:
data = [
    [('respCode', 'Exp_ResponseCode'), ('respDesc', 'Exp_ResponseDesc'),
     ('consumedLimit', 'Exp_ConsumedLimit')],
    {
        'Verify_Card_Balance': [['req_cardNo']],
        'Verify_Trans_Requests': [['given_transId'], ('remaining_balance', 'exp_remainingBalance'), ('available_balance', 'exp_availableBalance')],
        'Verify_Trans_Request1s': [['given_transId'], ('credit_limit_1', 'exp_creditLimit_1s'), ('cash_adv_limit', 'exp_cashAdvLimit'), ('card_balance', 'exp_cardBalance'), ('ledger_balance', 'exp_ledgerBalance')],
        'Verify_CreditCardAttrs': [['req_cardNo'], ('credit_limit_2', 'exp_creditLimit2'), ('ch_credit_limit', 'exp_chcreditLimit')]
    }
]

⭕ Queries

In [72]:
queries = {'Verify_Card_Balance': 'select * from card_funds where card_no = ?;',
           'Verify_Trans_Requests': 'select remaining_balance,available_balance from trans_requests where trace_audit_no = ?;',
           'Verify_Trans_Request1s': 'select credit_limit as credit_limit_1,cash_adv_limit,card_balance,ledger_balance from trans_request1s where trace_audit_no = ?;',
           'Verify_CreditCardAttrs': 'select credit_limit as credit_limit_2,ch_credit_limit from card_credit_attrs where card_srno in (select card_srno from cards where card_no = ?);'}

▶ Run All

In [73]:
def make_dbCheckpointMapping(data, queries):
  """returns dbCheckpointMapping"""
  s = ''
  d = {}
  for obj in data:
    if type(obj) == dict:
      d = obj
  for checkpoints in d:
    p  = [i for i in d[checkpoints] if type(i) == list]
    p = sum(p, [])
    meta = [i[1] for i in p if type(i) == tuple]
    p = [i[1] if type(i) == tuple else i for i in p]
    if p:
      s += f"""        {checkpoints}: {{
              query: '{queries[checkpoints]}',
              params: {p},
        """
      if meta:
        for m in meta:
          s += f"""  meta: {{
                {m}: SOURCE[0],
        """
      s += """},\n"""
  return s

In [74]:
def getonecheckpoint(l, key='enter db checkpoint', isdict=True):
    """returns string in expected format"""
    s = "" 
    actual = excepted = ""
    p  = [i for i in l if type(i) == list]
    p = sum(p, [])
    for el in l:
      actual, excepted = getactandexp(el)
      if not actual:
        continue
      s += f"""        {actual}: {{
          checkPointValueColumn: '{excepted}',"""
      if isdict:
        if p:
          s += f"""
          dbCheckPointName: '{key}'"""
        else:
          s += f"""
          outputCheckPointName: '{key}'"""
      s += """\n        },\n"""
    return s

In [75]:
def make_verificationMap(data):
  """retruns verficationMap"""
  s = ""
  for obj in data:
    l = []
    d = {}
    if type(obj) == list:
      l = obj
    elif type(obj) == dict:
      d = obj
    s += getonecheckpoint(l, isdict=False)
    for key in d:
      s += getonecheckpoint(d[key], key=key)
  return s

In [76]:
def getactandexp(obj):
  """return actual and expected"""
  actual = expected = ''
  if type(obj) == list:
    return None, None
  if type(obj) == tuple:
    actual, expected = obj
  else:
    actual = expected = obj
  return actual, expected

In [77]:
import numpy as np
def makecheckpoints(data):
  """return output and db check points"""
  def allcheckpoint(lis):
    """return all check point"""
    result = []
    for el in lis:
      actual, expected = getactandexp(el)
      if actual:
        result.append(actual)
    return result
  out, db = [], []
  for obj in data:
    l = []
    d = {}
    if type(obj) == list:
      out = allcheckpoint(obj)
    elif type(obj) == dict:
      for key in obj:
        db.append(allcheckpoint(obj[key]))
    db_result = []
    [db_result.append(j) for i in db for j in i]
  return out, db_result

In [78]:
def letsmake(api=apiname, start=start, end=end):
  """return config.js"""
  output, db = makecheckpoints(data)
  s = 'const SOURCES = ["res", "defaultSheet"];\n'
  s += f"""const config = {{
    apiName: "{api}",
    START_TEST_CASE: {start},
    END_TEST_CASE: {end},
    dbCheckpointNames: {db},
    outputCheckpointNames: {output},
    dbCheckpointMapping: {{
{make_dbCheckpointMapping(data, queries)}    }},
    verificationMap: {{
{make_verificationMap(data)}    }},
}};"""
  return s
s = letsmake()
s += f"""
export default config;"""
s

'const SOURCES = ["res", "defaultSheet"];\nconst config = {\n    apiName: "transactionHistoryMultiAccounts",\n    START_TEST_CASE: 0,\n    END_TEST_CASE: 9,\n    dbCheckpointNames: [\'remaining_balance\', \'available_balance\', \'credit_limit_1\', \'cash_adv_limit\', \'card_balance\', \'ledger_balance\', \'credit_limit_2\', \'ch_credit_limit\'],\n    outputCheckpointNames: [\'respCode\', \'respDesc\', \'consumedLimit\'],\n    dbCheckpointMapping: {\n        Verify_Card_Balance: {\n              query: \'select * from card_funds where card_no = ?;\',\n              params: [\'req_cardNo\'],\n        },\n        Verify_Trans_Requests: {\n              query: \'select remaining_balance,available_balance from trans_requests where trace_audit_no = ?;\',\n              params: [\'given_transId\'],\n        },\n        Verify_Trans_Request1s: {\n              query: \'select credit_limit as credit_limit_1,cash_adv_limit,card_balance,ledger_balance from trans_request1s where trace_audit_no = ?